# INLS 613 - Fake News Detection

Import libraries

In [1]:
import pandas as pd
import numpy as np
import scipy

## Dataset 1 - Train1.csv Exploration

In [2]:

data = pd.read_csv("train1.csv",encoding='utf_8') 




The data features are as follows:
- ID
- Title
- Author
- Text 
- Label (1 = "Unreliable", 0 = "Reliable")

We will only be using Title and Text as features to try and learn the concept of reliable vs unreliable news. 

Below is a sample set of the first 10 instances of data as an example: 

In [3]:
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",0
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,1
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",0
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,0
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0


In [4]:
data.shape

(20800, 5)

In [5]:
data['label'].unique()

array([1, 0])

In [6]:
data['label'].value_counts()

1    10413
0    10387
Name: label, dtype: int64

Replacing 1's and 0's with "Unreliable" and "Reliable" for clarity:

In [7]:
data['label'] = data['label'].replace(1, 'Unreliable')
data['label'] = data['label'].replace(0, 'Reliable')
data.head(10)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,Unreliable
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,Reliable
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",Unreliable
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,Unreliable
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,Unreliable
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi...",Reliable
6,6,Life: Life Of Luxury: Elton John’s 6 Favorite ...,NaN,Ever wonder how Britain’s most iconic pop pian...,Unreliable
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi...",Reliable
8,8,Excerpts From a Draft Script for Donald Trump’...,NaN,Donald J. Trump is scheduled to make a highly ...,Reliable
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,Reliable


In [57]:
data['text'].head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    Ever get the feeling your life circles the rou...
2    Why the Truth Might Get You Fired October 29, ...
3    Videos 15 Civilians Killed In Single US Airstr...
4    Print \nAn Iranian woman has been sentenced to...
Name: text, dtype: object

In [8]:
data['label'].value_counts()

Unreliable    10413
Reliable      10387
Name: label, dtype: int64

The dataset seems to have been read in appropriately and contains a near 50% split of labels, we can now begin the ML process. 

## Dataset 1 - Train1.csv - Machine Readable Conversion

In [9]:
from sklearn import preprocessing ### Importing a preprocessor to convert the labels in the target class.

In [10]:
train_class_y = ['Unreliable','Reliable']

In [11]:
le = preprocessing.LabelEncoder() ## Label encoder does the trick. 

In [12]:
le.fit(train_class_y) ## We are fitting the categories now. 

LabelEncoder()

In [13]:
train_y = le.transform(data['label'])

In [14]:
train_y ## These are out labels now. 

array([1, 0, 1, ..., 0, 1, 1])

In [15]:
le.transform(['Unreliable','Reliable','Reliable']) ### Just to check

array([1, 0, 0])

In [16]:
le.inverse_transform([0,1,1]) ### Doing an inverse of the transformation

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['Reliable', 'Unreliable', 'Unreliable'], dtype='<U10')

## Selecting Features Using TF.IDF

In [43]:
train_text = data['text']
train_title = data['title']
shape = [train_text.shape, train_title.shape]

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer ## Importing the library that will help us do this. 

In [66]:
vectorizer = TfidfVectorizer(min_df=1,stop_words='english',max_features=5000, ngram_range = (1,3)) ## Ask yourself, why min_df =1? We are using english stopwords. 
####max_features=3000 

In [67]:
train_text_tfidf = vectorizer.fit_transform(train_text.values.astype('U'))
#train_title_tfidf = vectorizer.fit_transform(train_title.values.astype('U'))

In [70]:
print(train_text)

0        House Dem Aide: We Didn’t Even See Comey’s Let...
1        Ever get the feeling your life circles the rou...
2        Why the Truth Might Get You Fired October 29, ...
3        Videos 15 Civilians Killed In Single US Airstr...
4        Print \nAn Iranian woman has been sentenced to...
5        In these trying times, Jackie Mason is the Voi...
6        Ever wonder how Britain’s most iconic pop pian...
7        PARIS  —   France chose an idealistic, traditi...
8        Donald J. Trump is scheduled to make a highly ...
9        A week before Michael T. Flynn resigned as nat...
10       Organizing for Action, the activist group that...
11       The BBC produced spoof on the “Real Housewives...
12       The mystery surrounding The Third Reich and Na...
13       Clinton Campaign Demands FBI Affirm Trump's Ru...
14       Yes, There Are Paid Government Trolls On Socia...
15       Guillermo Barros Schelotto was not the first A...
16       The scandal engulfing Wells Fargo toppled its .

In [68]:
print(vectorizer.get_feature_names())

['00', '000', '000 people', '10', '10 000', '10 percent', '10 years', '100', '100 000', '11', '12', '125', '13', '14', '15', '15 years', '150', '16', '17', '18', '19', '1960s', '1970s', '1980', '1980s', '1990', '1990s', '1991', '1992', '1994', '1995', '1996', '1998', '1999', '20', '20 percent', '20 years', '200', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2016 election', '2016 presidential', '2017', '2018', '2020', '20th', '21', '21st', '22', '23', '24', '25', '250', '26', '26 2016', '27', '27 2016', '28', '28 2016', '29', '29 2016', '30', '30 years', '300', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '400', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '50 000', '500', '51', '52', '53', '54', '55', '56', '58', '60', '600', '65', '70', '700', '75', '80', '800', '85', '8th', '90', '95', '99', '_____', 'aaron', 'abandoned', 'abc', 'abe', 'abedin', 'ability', 'able',

In [74]:
train_text_tfidf.shape

(20800, 5000)

In [85]:
print(train_text_tfidf)

  (0, 2168)	0.12940988721898655
  (0, 248)	0.20275661587045457
  (0, 1313)	0.18368632604293986
  (0, 924)	0.38172355987380774
  (0, 2564)	0.24280589565725202
  (0, 2382)	0.22159533126240064
  (0, 4659)	0.08612427622065416
  (0, 3048)	0.053067725710507004
  (0, 76)	0.05525700898032327
  (0, 54)	0.03829911233377042
  (0, 4338)	0.045622762252464706
  (0, 292)	0.019402119335482526
  (0, 4729)	0.09927652576724967
  (0, 2213)	0.03386503666050077
  (0, 1101)	0.09608642425926019
  (0, 2786)	0.03116586777118279
  (0, 455)	0.03207182350933723
  (0, 1114)	0.042392583324262205
  (0, 2578)	0.043082028357412955
  (0, 1409)	0.03515617999117226
  (0, 4945)	0.1440900976382808
  (0, 3242)	0.08072307444851475
  (0, 4937)	0.0574730396422326
  (0, 4869)	0.021831449461733245
  (0, 1733)	0.1983668987898422
  :	:
  (20799, 2307)	0.03290927627637119
  (20799, 4859)	0.062462064021677234
  (20799, 79)	0.02719898937613141
  (20799, 1283)	0.08054758947236221
  (20799, 4560)	0.28842165526045843
  (20799, 4618)	0.06

In [80]:
train_text_tfidf_array = train_text_tfidf.toarray()

In [87]:
train_text_tfidf_array[2]

array([0., 0., 0., ..., 0., 0., 0.])

In [84]:
vectorizer.inverse_transform(train_text_tfidf_array[0]) ## just to check what all features are there. 

[array(['2012', '2016', '25', '28', '28 2016', '30', 'according', 'acting',
        'actually', 'advance', 'agency', 'aide', 'american', 'announcing',
        'answer', 'anthony', 'anthony weiner', 'apparently', 'appear',
        'attempt', 'available', 'basic', 'boss', 'briefly', 'buy',
        'carolina', 'case', 'chairman', 'christian', 'classified',
        'classified information', 'click', 'clinton', 'clinton email',
        'colleagues', 'college', 'comey', 'committee', 'connect',
        'contained', 'control', 'cook', 'course', 'courtesy', 'creative',
        'daily', 'dan', 'democratic', 'democrats', 'desire', 'development',
        'didn', 'didn know', 'director', 'director james',
        'director james comey', 'discovered', 'district', 'does', 'doesn',
        'doing', 'don', 'don know', 'doubt', 'email', 'email server',
        'emails', 'entire', 'example', 'existence', 'explosive',
        'facebook', 'fact', 'facts', 'far', 'fbi', 'fbi director',
        'fbi director

In [90]:
print(train_text[0])

House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It By Darrell Lucus on October 30, 2016 Subscribe Jason Chaffetz on the stump in American Fork, Utah ( image courtesy Michael Jolley, available under a Creative Commons-BY license) 
With apologies to Keith Olbermann, there is no doubt who the Worst Person in The World is this week–FBI Director James Comey. But according to a House Democratic aide, it looks like we also know who the second-worst person is as well. It turns out that when Comey sent his now-infamous letter announcing that the FBI was looking into emails that may be related to Hillary Clinton’s email server, the ranking Democrats on the relevant committees didn’t hear about it from Comey. They found out via a tweet from one of the Republican committee chairmen. 
As we now know, Comey notified the Republican chairmen and Democratic ranking members of the House Intelligence, Judiciary, and Oversight committees that his agency was reviewing emails 

## Naive Bayes!

In [91]:
from sklearn.naive_bayes import MultinomialNB

In [92]:
mnb = MultinomialNB(alpha=1.0) # alpha is smoothing, setting to 1 is laplace smoothing